In [1]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
model_type = "whisper-small"
model_name = f"openai/{model_type}"

In [4]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    model_name, language="bengali", task="transcribe"
)

In [19]:
import pandas as pd
import os
from torch.utils.data import Dataset
import librosa
from datasets import Audio

class AudioDataset(Dataset):
    def __init__(self, labels_df: pd.DataFrame, data_path: str, processor):
        self.labels_df = labels_df
        self.data_path = data_path
        self.processor = processor
    def __len__(self):
        return len(self.labels_df)
    def __getitem__(self,idx):
        row = self.labels_df.iloc[idx]
        path = os.path.join(self.data_path, row["id"]+".mp3")
        sentence = row["sentence"]
        with open(path, "rb") as f:
            speech = f.read()
            audio = Audio(sampling_rate=processor.feature_extractor.sampling_rate).decode_example({"path": path, "bytes": speech})

        example = processor(audio=audio["array"], sampling_rate=processor.feature_extractor.sampling_rate, text=sentence)
        example["input_length"] = len(audio["array"]) // processor.feature_extractor.sampling_rate
        # speech, sr = librosa.load(path, sr=processor.feature_extractor.sampling_rate) 
#         print(speech.shape)
        return example

In [20]:
from datasets import DatasetDict
labels_df = pd.read_csv("bengaliai-speech/train.csv")

In [21]:
common_voice = DatasetDict()
train_dataset= AudioDataset(labels_df[labels_df["split"] == "train"], data_path="bengaliai-speech/train_mp3s", processor=processor)
test_dataset = AudioDataset(labels_df[labels_df["split"] == "valid"], data_path="bengaliai-speech/train_mp3s", processor=processor)

In [22]:
common_voice = DatasetDict({"train":train_dataset,"test": test_dataset})

In [23]:
from transformers.models.whisper.tokenization_whisper import TO_LANGUAGE_CODE

TO_LANGUAGE_CODE

{'english': 'en',
 'chinese': 'zh',
 'german': 'de',
 'spanish': 'es',
 'russian': 'ru',
 'korean': 'ko',
 'french': 'fr',
 'japanese': 'ja',
 'portuguese': 'pt',
 'turkish': 'tr',
 'polish': 'pl',
 'catalan': 'ca',
 'dutch': 'nl',
 'arabic': 'ar',
 'swedish': 'sv',
 'italian': 'it',
 'indonesian': 'id',
 'hindi': 'hi',
 'finnish': 'fi',
 'vietnamese': 'vi',
 'hebrew': 'he',
 'ukrainian': 'uk',
 'greek': 'el',
 'malay': 'ms',
 'czech': 'cs',
 'romanian': 'ro',
 'danish': 'da',
 'hungarian': 'hu',
 'tamil': 'ta',
 'norwegian': 'no',
 'thai': 'th',
 'urdu': 'ur',
 'croatian': 'hr',
 'bulgarian': 'bg',
 'lithuanian': 'lt',
 'latin': 'la',
 'maori': 'mi',
 'malayalam': 'ml',
 'welsh': 'cy',
 'slovak': 'sk',
 'telugu': 'te',
 'persian': 'fa',
 'latvian': 'lv',
 'bengali': 'bn',
 'serbian': 'sr',
 'azerbaijani': 'az',
 'slovenian': 'sl',
 'kannada': 'kn',
 'estonian': 'et',
 'macedonian': 'mk',
 'breton': 'br',
 'basque': 'eu',
 'icelandic': 'is',
 'armenian': 'hy',
 'nepali': 'ne',
 'mongol

In [24]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union



class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __init__(self, processor):
        self.processor = processor

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [25]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [26]:
import evaluate

metric = evaluate.load("wer")

In [27]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}


In [28]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [29]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="bengali", task="transcribe", use_cache=True
)

In [30]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{model_type}-bn",  # name on the HF Hub
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=4000,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    dataloader_num_workers=64,
)


In [31]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

/workspaces/HuggingFace-Audio-Course/./whisper-small-bn is already a clone of https://huggingface.co/Kurokabe/whisper-small-bn. Make sure you pull the latest changes with `repo.git_pull()`.


In [32]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


In [ ]:
kwargs = {
    "model_name": f"{model_type} bn",  # a 'pretty' name for your model
    "language": "bn",
     "dataset_tags": "Bengali, Kaggle competition",
    "finetuned_from": model_name,
    "tasks": "automatic-speech-recognition",
}
trainer.push_to_hub(**kwargs)

In [ ]:
model = model.cpu()

In [ ]:
predicted_ids = model.generate(common_voice["test"][0]["input_features"])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)


In [ ]:
transcription

['গরোমা প্রথম স্রিক্টি অত্র হয়ের প্রকেছেলেল।']